In [ ]:


import os
import math
import random
import argparse 
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

from sklearn.model_selection import train_test_split
from scipy.optimize import linear_sum_assignment


# Dataset (Updated)
class JigsawDataset(Dataset):
    """
    Assumes CSV with columns: "Image ID" (image file name) and "Label" (permutation string)
    """
    def __init__(self, csv_path, images_dir, image_size=201, transform=None):
        self.df = pd.read_csv(csv_path)
        self.images_dir = images_dir
        self.transform = transform
        self.image_size = image_size  
        assert image_size % 3 == 0, "image_size must be divisible by 3"

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        img_name = row['image']
        label = row['label']
        
        img_path = os.path.join(self.images_dir, img_name)
        img = Image.open(img_path).convert('RGB')
        img = img.resize((self.image_size, self.image_size), Image.BILINEAR)
        img = np.array(img).astype(np.float32) / 255.0
        cut = self.image_size // 3
        
        # tiles: list of (C,H,W)
        tiles = []
        for i in range(3):
            for j in range(3):
                tile = img[i*cut:(i+1)*cut, j*cut:(j+1)*cut, :]
                tile_img = Image.fromarray((tile * 255).astype(np.uint8))
                if self.transform:
                    tile_img = self.transform(tile_img)
                else:
                    transform = transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485,0.456,0.406],
                                             std=[0.229,0.224,0.225])
                    ])
                    tile_img = transform(tile_img)
                tiles.append(tile_img)  # C,H,W
        tiles = torch.stack(tiles)  # shape (9, C, H, W)

        label_list = [int(x) for x in str(label).split()]
        label_tensor = torch.tensor(label_list, dtype=torch.long)  # shape (9,)
        return tiles, label_tensor, img_name
    
# Model: per-tile encoder -> transformer -> per-tile logits over positions

class TileTransformer(nn.Module):
    def __init__(self, tile_encoder='resnet18', embed_dim=512, nhead=8, nlayers=3, dropout=0.1):
        super().__init__()
        res = models.resnet18(pretrained=True)
        res.fc = nn.Identity()
        self.backbone = res  #  512-d for resnet18
        self.embed_dim = 512
        if self.embed_dim != embed_dim:
            self.proj = nn.Linear(self.embed_dim, embed_dim)
            final_dim = embed_dim
        else:
            self.proj = None
            final_dim = self.embed_dim
        encoder_layer = nn.TransformerEncoderLayer(d_model=final_dim, nhead=nhead, dropout=dropout, dim_feedforward=final_dim*4, activation='gelu')
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=nlayers)
        self.head = nn.Linear(final_dim, 9)

    def forward(self, tiles):
        # tiles: (B, 9, C, H, W)
        B = tiles.shape[0]
        n = tiles.shape[1]  
        tiles_flat = tiles.view(B*n, tiles.shape[2], tiles.shape[3], tiles.shape[4])
        # run backbone per tile
        x = self.backbone(tiles_flat)  
        if self.proj is not None:
            x = self.proj(x)  
        x = x.view(B, n, -1)  # (B, 9, D)
        # transformer expects seq_len, batch, dim
        x_t = x.permute(1, 0, 2)
        x_out = self.transformer(x_t)  
        x_out = x_out.permute(1, 0, 2)  
        logits = self.head(x_out)  # per tile logits over positions
        return logits

def hungarian_from_logits(logits):
    """
    logits: numpy array shape (9,9) or tensor -> we compute assignment that maximizes sum of logit chosen.
    Returns a permutation list p of length 9 where p[tile_index] = assigned_position
    (tile i assigned to position p[i]).
    """
    if isinstance(logits, torch.Tensor):
        arr = logits.detach().cpu().numpy()
    else:
        arr = np.array(logits)
    cost = -arr
    row_ind, col_ind = linear_sum_assignment(cost)
    perm = [-1]*arr.shape[0]
    for r,c in zip(row_ind, col_ind):
        perm[r] = int(c)
    return perm

def compute_pra(pred_perm, true_perm):
    """pred_perm and true_perm are lists length 9, perm[i] = position for tile i"""
    return int(pred_perm == true_perm)

def compute_paa(pred_perm, true_perm):
    """Pairwise Adjacency Accuracy (4-neighbor). For each adjacent pair in grid positions,
       check if that adjacency is preserved in prediction.
       We'll compute count_correct_pairs / total_pairs.
    """
    inv_pred = {pos: i for i,pos in enumerate(pred_perm)}
    inv_true = {pos: i for i,pos in enumerate(true_perm)}
    neighbor_pairs = []
    for r in range(3):
        for c in range(3):
            pos = r*3 + c
            if c < 2:
                neighbor_pairs.append((pos, pos+1))
            if r < 2:
                neighbor_pairs.append((pos, pos+3))
    total = len(neighbor_pairs)
    correct = 0
    for a,b in neighbor_pairs:
        tile_a_true = inv_true[a]
        tile_b_true = inv_true[b]
        pred_pos_a = pred_perm[tile_a_true]
        pred_pos_b = pred_perm[tile_b_true]
        ra, ca = divmod(pred_pos_a, 3)
        rb, cb = divmod(pred_pos_b, 3)
        if abs(ra-rb) + abs(ca-cb) == 1:
            correct += 1
    return correct / total if total > 0 else 0.0

# Training & eval loops
def train_one_epoch(model, dataloader, opt, device):
    model.train()
    total_loss = 0.0
    for tiles, labels, _ in tqdm(dataloader, desc="train"):
        tiles = tiles.to(device) 
        labels = labels.to(device)  
        opt.zero_grad()
        logits = model(tiles)  
        B = logits.shape[0]
        logits_flat = logits.view(B*9, 9)
        labels_flat = labels.view(B*9)
        loss = F.cross_entropy(logits_flat, labels_flat)
        loss.backward()
        opt.step()
        total_loss += loss.item() * tiles.shape[0]
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, device):
    model.eval()
    total_pra = 0
    total_paa = 0.0
    n = 0
    with torch.no_grad():
        for tiles, labels, _ in tqdm(dataloader, desc="eval"):
            tiles = tiles.to(device)
            labels = labels.to(device)
            logits = model(tiles)  # (B,9,9)
            B = logits.shape[0]
            for i in range(B):
                perm = hungarian_from_logits(logits[i])
                true = labels[i].cpu().numpy().tolist()
                total_pra += compute_pra(perm, true)
                total_paa += compute_paa(perm, true)
                n += 1
    pra = total_pra / n
    paa = total_paa / n
    return pra, paa

# Inference helper for single image -> permutation string
def infer_single_image(model, image_path, image_size=201, device='cpu'):
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
    ])
    img = Image.open(image_path).convert('RGB')
    img = img.resize((image_size, image_size), Image.BILINEAR)
    img = np.array(img).astype(np.uint8)
    cut = image_size // 3
    tiles = []
    to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225])
    ])
    for i in range(3):
        for j in range(3):
            tile = img[i*cut:(i+1)*cut, j*cut:(j+1)*cut, :]
            tile_img = Image.fromarray(tile)
            tiles.append(to_tensor(tile_img))
    tiles = torch.stack(tiles).unsqueeze(0).to(device)  # (1,9,C,H,W)
    model.eval()
    with torch.no_grad():
        logits = model(tiles)[0]  # (9,9)
        perm = hungarian_from_logits(logits)
    return perm

# Main & arg parsing
def main(args):
    device = torch.device('cuda' if torch.cuda.is_available() and not args.force_cpu else 'cpu')
    print("Using device:", device)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(size=args.tile_size, scale=(0.8,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225])
    ])
    image_size = args.tile_size * 3

    ds = JigsawDataset(args.csv, args.images_dir, image_size=image_size, transform=None) # Pass None to use default
    
    
    idxs = list(range(len(ds)))
    train_idx, val_idx = train_test_split(idxs, test_size=args.val_frac, random_state=42)
    train_ds = torch.utils.data.Subset(ds, train_idx)
    val_ds = torch.utils.data.Subset(ds, val_idx)
    
    def collate_fn(batch):
        tiles = torch.stack([b[0] for b in batch], dim=0)  # (B,9,C,H,W)
        labels = torch.stack([b[1] for b in batch], dim=0)
        names = [b[2] for b in batch]
        
        return tiles, labels, names

    train_loader = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True, num_workers=2, collate_fn=collate_fn, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=args.batch_size, shuffle=False, num_workers=2, collate_fn=collate_fn, pin_memory=True)

    model = TileTransformer(tile_encoder='resnet18', embed_dim=512, nhead=8, nlayers=3).to(device)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.5)

    best_pra = -1.0
    for epoch in range(args.epochs):
        print(f"Epoch {epoch+1}/{args.epochs}")
        train_loss = train_one_epoch(model, train_loader, opt, device)
        pra, paa = evaluate(model, val_loader, device)
        print(f"Train loss: {train_loss:.4f}  Val PRA: {pra:.4f}  Val PAA: {paa:.4f}")
        if pra > best_pra:
            best_pra = pra
            torch.save(model.state_dict(), args.save_model)
            print("Saved best model:", args.save_model)
        scheduler.step()

    print(f"Training complete. Best model saved to {args.save_model} with PRA: {best_pra:.4f}")
    if args.test_image:
        print(f"Running inference on: {args.test_image}")
        model.load_state_dict(torch.load(args.save_model, map_location=device))
        perm = infer_single_image(model, args.test_image, image_size=image_size, device=device)
        print("Predicted permutation string:", " ".join(str(x) for x in perm))

In [ ]:

import argparse 

args = argparse.Namespace()

args.csv = "train.csv"
args.images_dir = "train/"
args.save_model = "best_jigsaw.pth"
args.test_image = ""  

args.tile_size = 67  

args.batch_size = 16
args.epochs = 100
args.lr = 1e-4
args.val_frac = 0.1

args.force_cpu = False

print("Configuration loaded:")
print(vars(args))

Configuration loaded:
{'csv': 'train.csv', 'images_dir': 'data/train/', 'save_model': 'best_jigsaw.pth', 'test_image': '', 'tile_size': 67, 'batch_size': 16, 'epochs': 100, 'lr': 0.0001, 'val_frac': 0.1, 'force_cpu': False}


In [ ]:
main(args)

Using device: cuda


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/100


train:   0%|          | 13/5240 [00:00<02:18, 37.66it/s]

eval: 100%|██████████| 583/583 [00:12<00:00, 45.77it/s]


...

Train loss: 0.0005  Val PRA: 0.9663  Val PAA: 0.9888
Epoch 99/100


eval: 100%|██████████| 583/583 [00:12<00:00, 47.89it/s]


Train loss: 0.0004  Val PRA: 0.9669  Val PAA: 0.9889
Epoch 100/100


eval: 100%|██████████| 583/583 [00:12<00:00, 48.02it/s]

Train loss: 0.0004  Val PRA: 0.9669  Val PAA: 0.9889
Training complete. Best model saved to best_jigsaw.pth with PRA: 0.9680


In [ ]:
# To predict the output run the following command:
"""
python code.py --image_dir valid \
                  --model_path best_jigsaw.pth \
                  --csv_output predictions.csv \
                  --json_output predictions.json
"""
# OR 
              
# python code.py --image_dir valid --model_path best_jigsaw.pth --csv_output predictions.csv --json_output predictions.json


import os
import csv
import json
import argparse
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from scipy.optimize import linear_sum_assignment


class TileTransformer(nn.Module):
    """
    Defines the model architecture. Must be identical to the
    class used during training to load the state_dict.
    """
    def __init__(self, tile_encoder='resnet18', embed_dim=512, nhead=8, nlayers=3, dropout=0.1):
        super().__init__()
        res = models.resnet18(pretrained=False) # Set pretrained=False for inference if not needed
        res.fc = nn.Identity()
        self.backbone = res  # outputs 512-d for resnet18
        self.embed_dim = 512
        if self.embed_dim != embed_dim:
            self.proj = nn.Linear(self.embed_dim, embed_dim)
            final_dim = embed_dim
        else:
            self.proj = None
            final_dim = self.embed_dim
        encoder_layer = nn.TransformerEncoderLayer(d_model=final_dim, nhead=nhead, dropout=dropout, dim_feedforward=final_dim*4, activation='gelu')
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=nlayers)
        self.head = nn.Linear(final_dim, 9)

    def forward(self, tiles):
        B = tiles.shape[0]
        n = tiles.shape[1]  # 9
        tiles_flat = tiles.view(B*n, tiles.shape[2], tiles.shape[3], tiles.shape[4])
        x = self.backbone(tiles_flat)  # (B*n, 512)
        if self.proj is not None:
            x = self.proj(x)  # (B*n, embed_dim)
        x = x.view(B, n, -1)  # (B, 9, D)
        # transformer expects seq_len, batch, dim
        x_t = x.permute(1, 0, 2)
        x_out = self.transformer(x_t)  
        x_out = x_out.permute(1, 0, 2)  
        logits = self.head(x_out)  #  per tile logits over positions
        return logits

def hungarian_from_logits(logits):

    if isinstance(logits, torch.Tensor):
        arr = logits.detach().cpu().numpy()
    else:
        arr = np.array(logits)
    cost = -arr
    row_ind, col_ind = linear_sum_assignment(cost)
    perm = [-1]*arr.shape[0]
    for r,c in zip(row_ind, col_ind):
        perm[r] = int(c)
    return perm

def infer_single_image(model, image_path, image_size=201, device='cpu'):

    to_tensor = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485,0.456,0.406],
                             std=[0.229,0.224,0.225])
    ])
    
    try:
        img = Image.open(image_path).convert('RGB')
        img = img.resize((image_size, image_size), Image.BILINEAR)
        img = np.array(img).astype(np.uint8)
    except Exception as e:
        print(f"Error opening or resizing image {image_path}: {e}")
        return None

    cut = image_size // 3
    tiles = []
    
    for i in range(3):
        for j in range(3):
            tile = img[i*cut:(i+1)*cut, j*cut:(j+1)*cut, :]
            tile_img = Image.fromarray(tile)
            tiles.append(to_tensor(tile_img))
            
    tiles = torch.stack(tiles).unsqueeze(0).to(device)  # (1, 9, C, H, W)
    
    model.eval()
    with torch.no_grad():
        logits = model(tiles)[0]  # (9, 9)
        perm = hungarian_from_logits(logits)
        
    return perm


def run_inference(model_path, image_dir, csv_output_path, json_output_path, tile_size, force_cpu):
    """
    Main function to run inference on a directory of images.
    """
    device = torch.device('cuda' if torch.cuda.is_available() and not force_cpu else 'cpu')
    print(f"Using device: {device}")

    image_size = tile_size * 3
    print(f"Using tile size: {tile_size}, full image size: {image_size}x{image_size}")

    print("Loading model...")
    model = TileTransformer(
        tile_encoder='resnet18', 
        embed_dim=512, 
        nhead=8, 
        nlayers=3
    ).to(device)
    
    try:
        model.load_state_dict(torch.load(model_path, map_location=device))
    except FileNotFoundError:
        print(f"Error: Model file not found at {model_path}")
        return
    except Exception as e:
        print(f"Error loading model state_dict: {e}")
        print("Ensure the model architecture in this script matches the one used for training.")
        return
        
    model.eval()
    print("Model loaded successfully.")

    image_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tif', '.tiff'}
    image_files = sorted([
        f for f in os.listdir(image_dir) 
        if os.path.splitext(f)[1].lower() in image_extensions
    ])
    
    if not image_files:
        print(f"No images found in directory: {image_dir}")
        return

    print(f"Found {len(image_files)} images to process...")

    all_results = []
    for filename in tqdm(image_files, desc="Predicting"):
        image_path = os.path.join(image_dir, filename)
        
        perm_list = infer_single_image(model, image_path, image_size=image_size, device=device)
        
        if perm_list:
            perm_string = " ".join(str(p) for p in perm_list)
            
            all_results.append({
                "filename": filename,
                "sequence_list": perm_list,
                "sequence_string": perm_string
            })
        else:
            print(f"Skipping corrupt or unreadable file: {filename}")

    print(f"Saving CSV results to {csv_output_path}...")
    try:
        with open(csv_output_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["filename", "sequence"])
            for res in all_results:
                writer.writerow([res["filename"], res["sequence_string"]])
    except Exception as e:
        print(f"Error saving CSV file: {e}")

    print(f"Saving JSON results to {json_output_path}...")
    json_data = {
        "images": [
            {
                "filename": res["filename"],
                "sequence": res["sequence_list"]
            } for res in all_results
        ]
    }
    
    try:
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(json_data, f, indent=2)
    except Exception as e:
        print(f"Error saving JSON file: {e}")
        
    print("\nInference complete.")
    print(f"CSV saved to: {csv_output_path}")
    print(f"JSON saved to: {json_output_path}")

# -----------------------------------------------------------

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Jigsaw Puzzle Inference Script")
    
    parser.add_argument(
        "--image_dir", 
        type=str, 
        required=True, 
        help="Path to the directory containing images for inference."
    )
    parser.add_argument(
        "--model_path", 
        type=str, 
        required=True, 
        help="Path to the saved model file (e.g., 'best_jigsaw.pth')."
    )
    parser.add_argument(
        "--csv_output", 
        type=str, 
        default="predictions.csv", 
        help="Path to save the output CSV file."
    )
    parser.add_argument(
        "--json_output", 
        type=str, 
        default="predictions.json", 
        help="Path to save the output JSON file."
    )
    parser.add_argument(
        "--tile_size", 
        type=int, 
        default=67, 
        help="Tile size used during training. (Default: 67, as in your script)"
    )
    parser.add_argument(
        "--force_cpu", 
        action='store_true', 
        help="Force use of CPU even if CUDA is available."
    )

    args = parser.parse_args()

    run_inference(
        model_path=args.model_path,
        image_dir=args.image_dir,
        csv_output_path=args.csv_output,
        json_output_path=args.json_output,
        tile_size=args.tile_size,
        force_cpu=args.force_cpu
    )